## Julia Primer 

This is a crash course on the Julia that you will need to know to complete the homeworks in this class. We are going to keep everything as simple as possible in the homeworks, oftentimes at the expense of speed. Because of this, we are going to be writing Julia in a way that is very similar to matlab. 

The great Brian Jackson (former TA of this course) made a fantastic [Julia Intro](https://github.com/Optimal-Control-16-745/JuliaIntro) that goes over everything one should know on their way to becoming a solid Julia programmer. This primer is simply a supplementary guide, and is not meant to replace the existing [Julia Intro](https://github.com/Optimal-Control-16-745/JuliaIntro). 

In [ ]:
# some basic path stuff

dir =  @__DIR__ # this is the path to the directory
dir_of_dir = dirname(@__DIR__) # this is the path to the directory of the directory
madeup_path = joinpath(dir, "made_up_file.jl")

@show dir 
@show dir_of_dir
@show madeup_path

In [ ]:
# show current environment (list out the packages in the environment with status)
import Pkg 
Pkg.status()

In [ ]:
# first we are going to activate a specific environment 
Pkg.activate(@__DIR__) # activate the env in the directory 

# now we can add a package if we want 
Pkg.add("BenchmarkTools")

In [ ]:
# we can also instantiate the environment 
Pkg.instantiate()      # download the packages listed in this environment 

In [ ]:
# standard library stuff (not env specific)
# import LinearAlgebra # LinearAlgebra.norm()
# import LinearAlegbra as la # la.norm()
using LinearAlgebra # norm()
using Test

## 1. General syntax and Linear Algebra

In [ ]:
a = 2 
b = 3.4
c = rand()  # uniform [0, 1]
d = randn() # gaussian [0, 1]

@show a 
@show a * b 
@show a ^ 2 
@show a + b; # jupyter always outputs the last thing in a cell, we can supress with ; 

In [ ]:
# julia people love using latex characters 
Ω = 4.5 # I did this with "\Omega<tab>"

# we can also add dots 
ẋ = 2.4; # I did this with "x\dot<tab>"

In [ ]:
# arrays 
vec = [1,2,3,4.0]  # vector, colons or commas work
mat = [1 2; 3 4.0] # matrix, must use colons between rows 

In [ ]:
vec[1:2] # indexing starts at 1 

In [ ]:
mat[:, 2] # all rows, 2nd column 

In [ ]:
# whenever I pull something 1 dimensional from a matrix, it will treat it as a vector
mat[2,1:2] # row 2, columns 1:2 (automatically converts to vector)

In [ ]:
reshape(mat[2,1:2], 1, 2) # if we want a matrix we can get it like this

In [ ]:
# I can edit these elements 
vec[3] = 100;
@show vec 

mat[1,1] = 20
@show mat

In [ ]:
# matrix multiplication
A = randn(2,2)
B = randn(2,2)
C = A * B 
D = A' * A # ' is transpose 

In [ ]:
# matrix vector multiplication 
A = randn(2,2)
b = randn(2)
c = A * b 
d = b' * A 

In [ ]:
size(A)

In [ ]:
size(b)

In [ ]:
# identity matrix is just I 
@show identity_2x2 = I(2) # we can specify a size 

@show zeros(2,2) + I # or we can just add it to something with I 

### 1.1 Solving Linear Systems

In [ ]:
# now let's solve some linear systems 
# A * x = b 

x = inv(A)*b # you SHOULD NOT solve linear systems like this 

x = A\b # you SHOULD solve them like this 

@test norm(A*x - b) < 1e-10 

In [ ]:
# matrix factorizations (advanced usage) 

# if I know something about the structure of my linear system, I can 
# choose how to factorize is 

A = randn(3,3); A = A'*A + I; # create a random positive definite matrix 
# this just means it will work for all factorizations

# cholesky is the fastest factorization, but only works on positive definite matrices
chol_factor = cholesky(A) # factorize into L*L'=A, where L is lower triangular
L = chol_factor.L # this is the lower triangular cholesky factor 
@test norm(L*L' - A) < 1e-10

# some other popular options that work for all full rank square matrices: 
qr_factor = qr(A)
Q = qr_factor.Q 
R = qr_factor.R 
@test norm(Q*R - A) < 1e-10

# LU decomposition 
lu_factor = lu(A)
L = lu_factor.L 
U = lu_factor.U 
@test norm(L*U - A) < 1e-10 

In [ ]:
# I can solve this linear system with these factors 
b = randn(3)

# solve x = A\b using our factors 
x1 = chol_factor\b 
x2 = qr_factor\b 
x3 = lu_factor\b 

@test norm(x1 - x2) < 1e-10 
@test norm(x1 - x3) < 1e-10 

In [ ]:
# ranges 
idx = 0:2:10                   # unit range
idx = range(0, 10, step = 2)   # same as above 
idx = range(0, 10, length = 6) # same as above 

## 2. Functions and Types

In [ ]:
# for loops 
for i = 1:2
    println("here is println output: ",i) # print line 
    @show i 
end

In [ ]:
# basic type information:
a = randn(2)::Vector{Float64}
@show typeof(a) # what type is a
@show eltype(a) # what element type is a 

In [ ]:
b = [1, 3]
@show typeof(b) # what type is a
@show eltype(b) # what element type is a 

In [ ]:
# some other types you will see 
A = randn(2,2)::Matrix{Float64} # this is a type assertion
B = [1 0; 0 3]::Matrix{Int64}
a = 4.3::Float64 

In [ ]:
randn(2,2)::Matrix{Int64} # type assertion will fail because randn creates floats

In [ ]:
# list comprehension
x = [i for i = 1:3]

In [ ]:
# we love vectors of vectors 
X = [randn(2) for i = 1:3]

In [ ]:
# we will use this for trajectories all the time, we can convert a vector of vectors 
# to a matrix with the following
X_matrix = hcat(X...) # this is the same as hcat(X[1],X[2],X[3])

In [ ]:
# we can also do vectors of matrices 
X = [randn(2,2) for i = 1:3]

In [ ]:
# functions 
function f(x, b) 
    return b*x 
end

output = f(randn(3), 2.4)

In [ ]:
# we can type the inputs and outputs to a function if we want to
function f2(x::Vector, b::Float64)::Matrix
    # this takes in the following:
    #     x - a vector (can be a vector of anything)
    #     b - a float 
    # and outputs a matrix (of anything)
    return b * x * x'
end

output = f2(randn(3), 2.4)

In [ ]:
# this is useful for 2 reasons:
# 1. it helps us avoid bugs by specifying the types we expect 
# 2. it allows for multiple dispatch

# here's what I mean by multiple dispatch
function print_my_type(a::Float64)
    println("thanks for inputting a ~float~")
end
print_my_type(3.4)

In [ ]:
# but if I give it an integer, it won't be able to find the function 
print_my_type(3)

In [ ]:
# this error is saying it doesn't have a method for `print_my_type(::Int64)`
# but the closest it found was `print_my_type(::Float64)`

# so let's write a new one 
function print_my_type(a::Int64)
    println("thanks for inputting an ~integer~")
end
print_my_type(3)

# these two functions have the same name, so the version that gets 
# called is determined by the type of the input 

## 2. Things to watch out for

In [ ]:
# global and local scope 

# everything so far has been in global scope, this means if I create a variable 
x = 4.6 

function myf()
    println("i'm printing x, even though it wasn't passed in")
    print("x: ",x)
end

myf()

In [ ]:
# this is super dangerous, and as a result global variables should NEVER be used 

# --------UNLESS------------
# we make it a const, which means we can't change it 

const x_const = 4.3

# now we should feel free to use it wherever

In [ ]:
# to avoid "polluting" our workspace with global variables, we often times 
# wrap things in a let end 

function test_local_scope()
    # since there are no inputs to this function, it will 
    # only know about global variables 
    
    println("here is global_var: ", global_var)
    println("here is local_var: ", local_var) # this will fail 
end

# global variable 
const global_var = 23

let 
    
    # local variable 
    local_var = 45 
    
    test_local_scope()
    
end

In [ ]:
# here is an example of global
x = 3.4 
function f1()
    if @isdefined(x)
        println("yeah I know about x")
        println("x: ", x)
    end
end
f1()

In [ ]:
# functions can modify their inputs in place
# this is not true in MATLAB, but is true in python 
function mutating_function!(input::Vector)
    input[2] = 140
    return nothing 
end

let 
    input = [5,9]
    println("input before: ", input)
    
    # call our function which modifies the input 
    mutating_function!(input)
    
    println("input after: ", input)
end

In [ ]:
# functions that modify the input have a ! at the end of the title 
x = [4, 1, 3, 2]
println("here is x: x = ", x)
println("here is the sorted version of x: ", sort(x))
println("but x is unchanged: x = ", x) 

# now I will sort the vector in place 
sort!(x) # ! means it will modify the input 
println("here is the modified x (it's sorted now): x = ", x)

In [ ]:
# we can apply any function to elements of an array with a . 
@show abs(3)
x = randn(4)
@show x
@show abs.(x)
X = randn(2,2)
@show X 
@show abs.(X)

In [ ]:
# even a vector of vectors 
X = [randn(2) for i = 1:3]

@show X
@show norm.(X)

## 3. Plotting

In [ ]:
# plotting 
using Plots 

# our first plot is just a normal one 
x = -10:.001:10 
plot(x, sin.(x), label = "sin(x)", xlabel = "x label",
     ylabel = "y label", xlim = (-5,5), ylim = (-2, 2))

In [ ]:
# now let's plot multiple 
plot(x, sin.(x), label = "sin(x)", xlabel = "x label",
     ylabel = "y label", xlim = (-5,5), ylim = (-2, 2))
# now I use plot! to modify my existing plot
plot!(x, cos.(x), label = "cos(x)")
scatter!(randn(100), randn(100), label = "random scatter points")

In [ ]:
# now let's plot a trajectory that's stored as vector of vectors 
time_vec = 0:.01:10 

# create a trajectory as a vector of vectors 
X = [[exp(-t), exp(-.5*t), exp(-.25*t)] for t in time_vec]

# convert this vector of vectors to a matrix 
Xmatrix = hcat(X...)

# plot this (DONT FORGET THE TRANSPOSE)
plot(time_vec, Xmatrix', label = ["exp(-t)" "exp(-.5t)" "exp(-.25t)"],
     ylabel = "ylabel", xlabel = "time (s)")

In [ ]:
import ForwardDiff as FD

function first_order_taylor(f::Function, x̄::Float64, x::Float64)::Float64
    # first order taylor series of f, 
    return f(x̄) + FD.derivative(f,x̄)*(x - x̄)
end

let 
    
    x = -6:.01:6
    y = sin.(x)
    
    # let's linearize about a random point 
    x̄ = randn()

    y_taylor = [first_order_taylor(sin, x̄, x[i]) for i = 1:length(x)]
    # y_taylor = [first_order_taylor(sin, x̄, xi) for xi in x] # you can also do this 
    
    scatter([x̄], [sin(x̄)], label = "linearization point")
    plot!(x, y_taylor, label = "1st order taylor series")
    display(plot!(x,y, label = "sin(x)", xlabel = "x", ylabel = "y"))
    
end